In [ ]:
###REMEMBER TO CLEAR os.chdir() AND RESTART KERNEL AND CLEAR OUTPUT 

###FOR EXAMPLE: AAPL
#https://investor.apple.com/sec-filings/default.aspx
#10Q and 10K in whichStock directory   

In [ ]:
from urllib.request import Request, urlopen

import os
import requests
import re
import json
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
#get API key file contents and connect to FMP API
os.chdir('~')

FMP_API_KEY = ""
BASE_URL = ""

with open('FMP_API.txt', 'r') as file: 
    for line in file: 
        if line.startswith('FMP_API_KEY'): 
            FMP_API_KEY = line.split('"')[1]
        elif line.startswith('BASE_URL'): 
            BASE_URL = line.split('"')[1]

ticker = input("Enter ticker symbol for Buffett DCA Analysis: ")

In [ ]:
# without limit parameter in URL: 
yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

In [ ]:
#[INCOME STATEMENT][REVENUE]
#   calculate gross profit, yearly and monthly
#   WB formula:   NET EARNINGS = TOTAL REVENUE - COGS
#   FMP formula:  REVENUE - COST OF REVENUE

yearly_metrics = pd.DataFrame()
yearly_metrics['Frequency'], yearly_metrics['Year'], yearly_metrics["Filing Date"], yearly_metrics["Net Earnings"] = ['', yearly_income_stmt['calendarYear'], yearly_income_stmt["fillingDate"], yearly_income_stmt['revenue'] - yearly_income_stmt['costOfRevenue']]
yearly_metrics.fillna('Annually', inplace=True)

#yearly_metrics

monthly_metrics = pd.DataFrame()
monthly_metrics['Frequency'], monthly_metrics['Year'], monthly_metrics['Quarter'], monthly_metrics["Filing Date"], monthly_metrics["Net Earnings"] = ['', monthly_income_stmt['calendarYear'], monthly_income_stmt['period'], monthly_income_stmt['fillingDate'], monthly_income_stmt['revenue'] - monthly_income_stmt['costOfRevenue']]
monthly_metrics.fillna('Quarterly', inplace = True)

#monthly_metrics


In [ ]:
#API response URLs with limit parameter in URL
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?limit=120&apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&limit=120&apikey=' + FMP_API_KEY)).json())

#without limit parameter in URL: 
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

#response_from_API = yearly_income.json()
#df = pd.DataFrame(response_from_API)